In [15]:
import pandas as pd
from megadetector.detection.run_detector_batch import load_and_run_detector_batch, write_results_to_file
from megadetector.utils import path_utils
import os
import pandas as pd
import io
import sys


# Define the directory
empty_folder = "../data/detector_tests/downloaded_empty_images"

# Get the list of files in the directory
file_names = os.listdir(empty_folder)

# Sort the file names numerically
file_names_sorted = sorted(file_names, key=lambda x: int(os.path.splitext(x)[0]))

# Create a DataFrame
df = pd.DataFrame(file_names_sorted, columns=["File Name"])

# Recursively find images in the processed folder

image_file_names = path_utils.find_images(empty_folder, recursive=True)

sys.stdout = io.StringIO()

confidence_threshold = 0.005

# Run MegaDetector, default confidence threshold 0.005
results = load_and_run_detector_batch('MDV5A', image_file_names, confidence_threshold)

# Initialize columns for category and confidence
df['Category'] = None
df['Confidence'] = None

# Dictionary to map category numbers to names
category_map = {'1': 'Animal', '2': 'Human', '3': 'Vehicle'}

# Populate the DataFrame
for entry in results:
    file_name = os.path.basename(entry['file'])
    max_confidence = entry['max_detection_conf']
    if entry['detections']:
        highest_conf_detection = max(entry['detections'], key=lambda x: x['conf'])
        category = category_map.get(highest_conf_detection['category'], 'Unknown')
    else:
        category = 'Empty'
    df.loc[df['File Name'] == file_name, 'Category'] = category
    df.loc[df['File Name'] == file_name, 'Confidence'] = max_confidence

df.to_csv('../data/detector_tests/data.csv', index=False)


,File Name
0,0.jpg
1,1.jpg
2,2.jpg
3,3.jpg
4,4.jpg
...,...
4831,4832.jpg
4832,4833.jpg
4833,4834.jpg
4834,4835.jpg


In [19]:
df

,File Name,Category,Confidence
0,0.jpg,Animal,0.0969
1,1.jpg,Animal,0.0182
2,2.jpg,Empty,0.0
3,3.jpg,Animal,0.325
4,4.jpg,Animal,0.00566
...,...,...,...
4831,4832.jpg,None,None
4832,4833.jpg,None,None
4833,4834.jpg,None,None
4834,4835.jpg,None,None
